In [117]:
import random
from binascii import a2b_hex, b2a_hex
import hashlib
from Crypto.Cipher import AES

In [118]:
# 快速幂取余算法 求(a ^ b) % n
def fast_exp_mod(a, b, n):
    res = 1
    while b != 0:
        if b & 1:
            res = (res * a) % n
        b >>= 1
        a = pow(a, 2, n)
    return res

# 测试
print(fast_exp_mod(2, 10, 11))
print(fast_exp_mod(11, 15, 13))

1
5


In [119]:
# 读文件
def read_file(path):
    with open(path, 'r') as f:
        content = f.read()
    f.close()
    return content

In [120]:
# 写文件
def write_file(content, path):
    with open(path, 'w') as f:
        f.write(content)
    f.close()

In [121]:
# 加密
def RSA_Enc(pk, m): 
    c = fast_exp_mod(m, pk[1], pk[0]) # c = (m ^ e) % N
    return c

In [122]:
# 解密
def RSA_Dec(sk, c): 
    m = fast_exp_mod(c, sk[1], sk[0]) # m = (c ^ d) % N
    return m

In [123]:
# OAEP-RSA加密
def OAEP_RSA_Enc(m, pk, k0=512):
    # 字符串转ascii码
    encoded_m = hex(int(m.encode('utf-8').hex(), 16))
    int_m = int(encoded_m, 16)

    # 计算k1，注意填充后结果不能超过N
    k1 = pk[0].bit_length() - int_m.bit_length() - k0 - 1
    
    # OAEP填充
    r = random.randrange(1 << (k0 - 1), (1 << k0) - 1)
    write_file(str(r), 'Random_Number.txt')
    g_r = hashlib.sha512(hex(r).encode('utf-8'))
    x = int(int_m << k1) ^ int(g_r.hexdigest(), 16)
    h_x = hashlib.sha512(hex(x).encode('utf-8'))
    y = r ^ int(h_x.hexdigest(), 16)
    padded_m = (x << k0) + y
    write_file(hex(padded_m), 'Message_After_Padding.txt')

    # RSA公钥加密填充后结果
    return RSA_Enc(pk, padded_m), k1

In [124]:
# OAEP-RSA解密
def OAEP_RSA_Dec(c, sk, k1, k0=512):
    # RSA私钥解密填充后结果
    padded_m = RSA_Dec(sk, c)

    # OAEP还原
    y = padded_m % (1 << k0)
    x = padded_m >> k0
    h_x = hashlib.sha512(hex(x).encode('utf-8'))
    r = y ^ int(h_x.hexdigest(), 16)
    g_r = hashlib.sha512(hex(r).encode('utf-8'))
    int_m = x ^ int(g_r.hexdigest(), 16)
    
    # ascii码转字符串
    encoded_m = a2b_hex(hex(int_m >> k1)[2:])
    m = str(encoded_m, encoding='utf-8')

    return m

In [125]:
# 随机种子
random.seed(520030910281)

# RSA公钥
pk = eval(read_file('RSA_Public_Key.txt'))
# RSA私钥
sk = eval(read_file('RSA_Secret_Key.txt'))

# 明文消息
plain_msg = "Sample Message"
print('Raw Message:', plain_msg)

# 加密
cipher_msg, k1 = OAEP_RSA_Enc(plain_msg, pk)
print('Encrypted Message:', hex(cipher_msg))
write_file(hex(cipher_msg), 'Encrypted_Message.txt')

# 解密
decipher_msg = OAEP_RSA_Dec(cipher_msg, sk, k1)
print("Decrypted Message:", decipher_msg)

Raw Message: Sample Message
Encrypted Message: 0x359b53c11e93c51f0677c0890a567d8c98053bd511a2263126d415fa5cca2fa6381e4f6ed7e0ad02ffd81d53753dae411702ada8c9535d7601a04d7bd0732185ef90f4769e0bea0c9e5e075e62164f6e506fa5baa5e803f16a2354c77a4a07a97557ef6901f6c82d80916f0af09b2ffcd455986d44a1bd0019c8df65a24cd96b
Decrypted Message: Sample Message
